In [5]:
from __future__ import print_function
from time import time

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups
from keras.models import Sequential
import numpy as np
import pickle
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import pandas as pd
# from Db import MongoTxA, IdQuery

with open('../datasets/Clasificacion-SegundoTexto.pickle', 'rb') as handle:
    data = pickle.load(handle)
    print("Data Loaded")
    
from sklearn.preprocessing import LabelEncoder

data_samples = data.cleanText
data_samples.fillna(value='vacío')
data.head()

Data Loaded


,_id,anno,cleanText,control_garantia,dia,expediente,fecha,mes,mesExtra,originalId,portanto,resultado,subtema,subtema_original,tema,termino,termino_original,tipo,voto,votoSalvado
0,5d01799d9cd40241709406f5,2013,expedient co expedient co exp co res recurs am...,GARANTIA,8,120141730007CO,1/8/2013,1,Jan-13,5d097311ef0c083cb4882823,1) Sentencia 2013-00001. Expediente 12-...,INADMISIBLE,FALTA DE RESPUESTA,FALTA DE RESPUESTA,PETICION,RP,RP,RA,000001-13,VS
1,5d01799d9cd40241709406f6,2013,exp co res sal constitucional cort suprem just...,GARANTIA,8,120146550007CO,1/8/2013,1,Jan-13,5d097311ef0c083cb4882824,2) Sentencia 2013-0000...,FONDO,MEDICAMENTO,MEDICAMENTO,SALUD,CL,CL,RA,000002-13,NO
2,5d01799d9cd40241709406f7,2013,exp co res sal constitucional cort suprem just...,GARANTIA,8,120149400007CO,1/8/2013,1,Jan-13,5d097311ef0c083cb4882825,3) Sentencia 2013-00...,FONDO,CITA MEDICA,CITA,SALUD,SIN LUGAR,SL,RA,000003-13,NO
3,5d01799d9cd40241709406f8,2013,exp co res sal constitucional cort suprem just...,GARANTIA,8,120154330007CO,1/8/2013,1,Jan-13,5d097311ef0c083cb4882826,4) Sentencia 2013-0000...,FONDO,ATESTADOS,ATESTADOS,TRABAJO,CL,CL,RA,000004-13,VS
4,5d01799d9cd40241709406f9,2013,exp co res sal constitucional cort suprem just...,GARANTIA,8,120154350007CO,1/8/2013,1,Jan-13,5d097311ef0c083cb4882827,5) Sentencia 2013-00005. Expediente 12-0...,FONDO,ATESTADOS,ATESTADOS,TRABAJO,CL,CL,RA,000005-13,VS


In [3]:
n_features = 1000
n_components = 10
n_top_words = 20


def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "\n\nTemas #%d: " % topic_idx
        message += "\n1\t".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()



vectorizer = CountVectorizer(max_df=0.95, min_df=2,max_features=4000)
corpus_vec = vectorizer.fit_transform(data_samples)
idf = TfidfTransformer()
tfidf = idf.fit_transform(corpus_vec)


In [ ]:
# Fit the NMF model
print("Ajustando el modelo ...")
t0 = time()
nmf = NMF(n_components=n_components, random_state=1,
          alpha=.1, l1_ratio=.5).fit(tfidf)
print("done in %0.3fs." % (time() - t0))

print("\nTemas  NMF")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)

In [ ]:
# Fit the NMF model
print("Ajustando NMF2")
t0 = time()
nmf = NMF(n_components=n_components, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(tfidf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in NMF model (generalized Kullback-Leibler divergence):")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)

In [6]:
n_components = 10
print("Fitting LDA models with tf features")
lda = LatentDirichletAllocation(n_components=n_components, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
t0 = time()
lda.fit(tfidf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in LDA model:")
tf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Fitting LDA models with tf features
done in 132.985s.

Topics in LDA model:


NameError: name 'tfidf_vectorizer' is not defined

In [ ]:
from sklearn.manifold import TSNE
from MulticoreTSNE import MulticoreTSNE as TSNE2
from tsnecuda import TSNE as TSNECUDA

# a t-SNE model
# angle value close to 1 means sacrificing accuracy for speed
# pca initializtion usually leads to better results 
tsne_model = TSNE(n_components=3, n_iter=500, perplexity=8 ,verbose=1, random_state=0, init='pca')

tsne_model2 = TSNE2(n_jobs=-1,n_components=3, n_iter=300, perplexity=50 ,verbose=1, random_state=0,  init='random')

#tsne_model3 = TSNECUDA(n_components=2, n_iter=300, perplexity=8 ,verbose=1, init='random')

tsne_lda =  tsne_model2.fit_transform(tfidf.toarray() ) 
# 20-D -> 2-D
#tsne_lda = tsne_lda = tsne_model.fit_transform(tfidf.toarray() ) 

import numpy as np
import bokeh.plotting as bp
from bokeh.plotting import save
from bokeh.models import HoverTool
from bokeh.io import output_notebook

n_top_words = 5 # number of keywords we show

# 20 colors
colormap = np.array([
    "#1f77b4", "#aec7e8", "#ff7f0e", "#ffbb78", "#2ca02c",
    "#98df8a", "#d62728", "#ff9896", "#9467bd", "#c5b0d5",
    "#8c564b", "#c49c94", "#e377c2", "#f7b6d2", "#7f7f7f",
    "#c7c7c7", "#bcbd22", "#dbdb8d", "#17becf", "#9edae5",
    "#8c564b", "#c49c94", "#e377c2", "#f7b6d2", "#7f7f7f",
    "#c7c7c7", "#bcbd22", "#dbdb8d", "#17becf", "#9edae5",
])

title = '20 newsgroups LDA viz'
num_example = len(data)

In [ ]:
from tsnecuda import TSNE as TSNECUDA

# a t-SNE model
# angle value close to 1 means sacrificing accuracy for speed
# pca initializtion usually leads to better results 
tsne_model3 = TSNECUDA(n_components=2, n_iter=300, perplexity=50 ,verbose=1, init='random')

tsne_lda2 =  tsne_model3.fit_transform(tfidf.toarray() ) 
# 20-D -> 2-D
#tsne_lda = tsne_lda = tsne_model.fit_transform(tfidf.toarray() ) 


In [ ]:
#y = lda.transform(tfidf)
#data['groups'] = y.argmax(1)
colormap = [
'#1abc9c', '#e8f8f5', '#d1f2eb', '#a3e4d7', '#76d7c4', '#48c9b0', '#1abc9c', '#17a589', 
'#148f77', '#117864', '#0e6251', '#16a085', '#e8f6f3', '#d0ece7', '#a2d9ce', '#73c6b6', 
'#45b39d', '#16a085', '#138d75', '#117a65', '#0e6655', '#0b5345', '#2ecc71', '#eafaf1', 
'#d5f5e3', '#abebc6', '#82e0aa', '#58d68d', '#2ecc71', '#28b463', '#239b56', '#1d8348', 
'#186a3b', '#27ae60', '#e9f7ef', '#d4efdf', '#a9dfbf', '#7dcea0', '#52be80', '#27ae60', 
'#229954', '#1e8449', '#196f3d', '#145a32', '#3498db', '#ebf5fb', '#d6eaf8', '#aed6f1', 
'#85c1e9', '#5dade2', '#3498db', '#2e86c1', '#2874a6', '#21618c', '#1b4f72', '#2980b9', 
'#eaf2f8', '#d4e6f1', '#a9cce3', '#7fb3d5', '#5499c7', '#2980b9', '#2471a3', '#1f618d', 
'#1a5276', '#154360', '#9b59b6', '#f5eef8', '#ebdef0', '#d7bde2', '#c39bd3', '#af7ac5', 
'#9b59b6', '#884ea0', '#76448a', '#633974', '#512e5f', '#8e44ad', '#f4ecf7', '#e8daef', 
'#d2b4de', '#bb8fce', '#a569bd', '#8e44ad', '#7d3c98', '#6c3483', '#5b2c6f', '#4a235a', 
'#34495e', '#ebedef', '#d6dbdf', '#aeb6bf', '#85929e', '#5d6d7e', '#34495e', '#2e4053', 
'#283747', '#212f3c', '#1b2631', '#2c3e50', '#eaecee', '#d5d8dc', '#abb2b9', '#808b96', 
'#566573', '#2c3e50', '#273746', '#212f3d', '#1c2833', '#17202a', '#f1c40f', '#fef9e7', 
'#fcf3cf', '#f9e79f', '#f7dc6f', '#f4d03f', '#f1c40f', '#d4ac0d', '#b7950b', '#9a7d0a', 
'#7d6608', '#f39c12', '#fef5e7', '#fdebd0', '#fad7a0', '#f8c471', '#f5b041', '#f39c12', 
'#d68910', '#b9770e', '#9c640c', '#7e5109', '#e67e22', '#fdf2e9', '#fae5d3', '#f5cba7', 
'#f0b27a', '#eb984e', '#e67e22', '#ca6f1e', '#af601a', '#935116', '#784212', '#d35400', 
'#fbeee6', '#f6ddcc', '#edbb99', '#e59866', '#dc7633', '#d35400', '#ba4a00', '#a04000', 
'#873600', '#6e2c00', '#e74c3c', '#fdedec', '#fadbd8', '#f5b7b1', '#f1948a', '#ec7063',
'#e74c3c', '#cb4335', '#b03a2e', '#943126', '#78281f', '#c0392b', '#f9ebea', '#f2d7d5',
'#e6b0aa', '#d98880', '#cd6155', '#c0392b', '#a93226', '#922b21', '#7b241c', '#641e16', 
'#ecf0f1', '#fdfefe', '#fbfcfc', '#f7f9f9', '#f4f6f7', '#f0f3f4', '#ecf0f1', '#d0d3d4', 
'#b3b6b7', '#979a9a', '#7b7d7d', '#bdc3c7', '#f8f9f9', '#f2f3f4', '#e5e7e9', '#d7dbdd', 
'#cacfd2', '#bdc3c7', '#a6acaf', '#909497', '#797d7f', '#626567', '#95a5a6', '#f4f6f6', 
'#eaeded', '#d5dbdb', '#bfc9ca', '#aab7b8', '#95a5a6', '#839192', '#717d7e', '#5f6a6a', 
'#4d5656', '#7f8c8d', '#f2f4f4', '#e5e8e8', '#ccd1d1', '#b2babb', '#99a3a4', '#7f8c8d',
 '#707b7c', '#616a6b', '#515a5a', '#424949', '#1abc9c', '#e8f8f5', '#d1f2eb', '#a3e4d7', 
    '#76d7c4', '#48c9b0', '#1abc9c', '#17a589', 
'#148f77', '#117864', '#0e6251', '#16a085', '#e8f6f3', '#d0ece7', '#a2d9ce', '#73c6b6', 
'#45b39d', '#16a085', '#138d75', '#117a65', '#0e6655', '#0b5345', '#2ecc71', '#eafaf1', 
'#d5f5e3', '#abebc6', '#82e0aa', '#58d68d', '#2ecc71', '#28b463', '#239b56', '#1d8348', 
'#186a3b', '#27ae60', '#e9f7ef', '#d4efdf', '#a9dfbf', '#7dcea0', '#52be80', '#27ae60', 
'#229954', '#1e8449', '#196f3d', '#145a32', '#3498db', '#ebf5fb', '#d6eaf8', '#aed6f1', 
'#85c1e9', '#5dade2', '#3498db', '#2e86c1', '#2874a6', '#21618c', '#1b4f72', '#2980b9', 
'#eaf2f8', '#d4e6f1', '#a9cce3', '#7fb3d5', '#5499c7', '#2980b9', '#2471a3', '#1f618d', 
'#1a5276', '#154360', '#9b59b6', '#f5eef8', '#ebdef0', '#d7bde2', '#c39bd3', '#af7ac5', 
'#9b59b6', '#884ea0', '#76448a', '#633974', '#512e5f', '#8e44ad', '#f4ecf7', '#e8daef', 
'#d2b4de', '#bb8fce', '#a569bd', '#8e44ad', '#7d3c98', '#6c3483', '#5b2c6f', '#4a235a', 
'#34495e', '#ebedef', '#d6dbdf', '#aeb6bf', '#85929e', '#5d6d7e', '#34495e', '#2e4053', 
'#283747', '#212f3c', '#1b2631', '#2c3e50', '#eaecee', '#d5d8dc', '#abb2b9', '#808b96', 
'#566573', '#2c3e50', '#273746', '#212f3d', '#1c2833', '#17202a', '#f1c40f', '#fef9e7', 
'#fcf3cf', '#f9e79f', '#f7dc6f', '#f4d03f', '#f1c40f', '#d4ac0d', '#b7950b', '#9a7d0a', 
'#7d6608', '#f39c12', '#fef5e7', '#fdebd0', '#fad7a0', '#f8c471', '#f5b041', '#f39c12', 
'#d68910', '#b9770e', '#9c640c', '#7e5109', '#e67e22', '#fdf2e9', '#fae5d3', '#f5cba7', 
'#f0b27a', '#eb984e', '#e67e22', '#ca6f1e', '#af601a', '#935116', '#784212', '#d35400', 
'#fbeee6', '#f6ddcc', '#edbb99', '#e59866', '#dc7633', '#d35400', '#ba4a00', '#a04000', 
'#873600', '#6e2c00', '#e74c3c', '#fdedec', '#fadbd8', '#f5b7b1', '#f1948a', '#ec7063',
'#e74c3c', '#cb4335', '#b03a2e', '#943126', '#78281f', '#c0392b', '#f9ebea', '#f2d7d5',
'#e6b0aa', '#d98880', '#cd6155', '#c0392b', '#a93226', '#922b21', '#7b241c', '#641e16', 
'#ecf0f1', '#fdfefe', '#fbfcfc', '#f7f9f9', '#f4f6f7', '#f0f3f4', '#ecf0f1', '#d0d3d4', 
'#b3b6b7', '#979a9a', '#7b7d7d', '#bdc3c7', '#f8f9f9', '#f2f3f4', '#e5e7e9', '#d7dbdd', 
'#cacfd2', '#bdc3c7', '#a6acaf', '#909497', '#797d7f', '#626567', '#95a5a6', '#f4f6f6', 
'#eaeded', '#d5dbdb', '#bfc9ca', '#aab7b8', '#95a5a6', '#839192', '#717d7e', '#5f6a6a', 
'#4d5656', '#7f8c8d', '#f2f4f4', '#e5e8e8', '#ccd1d1', '#b2babb', '#99a3a4', '#7f8c8d',
 '#707b7c', '#616a6b', '#515a5a', '#424949',
    '#1abc9c', '#e8f8f5', '#d1f2eb', '#a3e4d7', '#76d7c4', '#48c9b0', '#1abc9c', '#17a589', 
'#148f77', '#117864', '#0e6251', '#16a085', '#e8f6f3', '#d0ece7', '#a2d9ce', '#73c6b6', 
'#45b39d', '#16a085', '#138d75', '#117a65', '#0e6655', '#0b5345', '#2ecc71', '#eafaf1', 
'#d5f5e3', '#abebc6', '#82e0aa', '#58d68d', '#2ecc71', '#28b463', '#239b56', '#1d8348', 
'#186a3b', '#27ae60', '#e9f7ef', '#d4efdf', '#a9dfbf', '#7dcea0', '#52be80', '#27ae60', 
'#229954', '#1e8449', '#196f3d', '#145a32', '#3498db', '#ebf5fb', '#d6eaf8', '#aed6f1', 
'#85c1e9', '#5dade2', '#3498db', '#2e86c1', '#2874a6', '#21618c', '#1b4f72', '#2980b9', 
'#eaf2f8', '#d4e6f1', '#a9cce3', '#7fb3d5', '#5499c7', '#2980b9', '#2471a3', '#1f618d', 
'#1a5276', '#154360', '#9b59b6', '#f5eef8', '#ebdef0', '#d7bde2', '#c39bd3', '#af7ac5', 
'#9b59b6', '#884ea0', '#76448a', '#633974', '#512e5f', '#8e44ad', '#f4ecf7', '#e8daef', 
'#d2b4de', '#bb8fce', '#a569bd', '#8e44ad', '#7d3c98', '#6c3483', '#5b2c6f', '#4a235a', 
'#34495e', '#ebedef', '#d6dbdf', '#aeb6bf', '#85929e', '#5d6d7e', '#34495e', '#2e4053', 
'#283747', '#212f3c', '#1b2631', '#2c3e50', '#eaecee', '#d5d8dc', '#abb2b9', '#808b96', 
'#566573', '#2c3e50', '#273746', '#212f3d', '#1c2833', '#17202a', '#f1c40f', '#fef9e7', 
'#fcf3cf', '#f9e79f', '#f7dc6f', '#f4d03f', '#f1c40f', '#d4ac0d', '#b7950b', '#9a7d0a', 
'#7d6608', '#f39c12', '#fef5e7', '#fdebd0', '#fad7a0', '#f8c471', '#f5b041', '#f39c12', 
'#d68910', '#b9770e', '#9c640c', '#7e5109', '#e67e22', '#fdf2e9', '#fae5d3', '#f5cba7', 
'#f0b27a', '#eb984e', '#e67e22', '#ca6f1e', '#af601a', '#935116', '#784212', '#d35400', 
'#fbeee6', '#f6ddcc', '#edbb99', '#e59866', '#dc7633', '#d35400', '#ba4a00', '#a04000', 
'#873600', '#6e2c00', '#e74c3c', '#fdedec', '#fadbd8', '#f5b7b1', '#f1948a', '#ec7063',
'#e74c3c', '#cb4335', '#b03a2e', '#943126', '#78281f', '#c0392b', '#f9ebea', '#f2d7d5',
'#e6b0aa', '#d98880', '#cd6155', '#c0392b', '#a93226', '#922b21', '#7b241c', '#641e16', 
'#ecf0f1', '#fdfefe', '#fbfcfc', '#f7f9f9', '#f4f6f7', '#f0f3f4', '#ecf0f1', '#d0d3d4', 
'#b3b6b7', '#979a9a', '#7b7d7d', '#bdc3c7', '#f8f9f9', '#f2f3f4', '#e5e7e9', '#d7dbdd', 
'#cacfd2', '#bdc3c7', '#a6acaf', '#909497', '#797d7f', '#626567', '#95a5a6', '#f4f6f6', 
'#eaeded', '#d5dbdb', '#bfc9ca', '#aab7b8', '#95a5a6', '#839192', '#717d7e', '#5f6a6a', 
'#4d5656', '#7f8c8d', '#f2f4f4', '#e5e8e8', '#ccd1d1', '#b2babb', '#99a3a4', '#7f8c8d',
 '#707b7c', '#616a6b', '#515a5a', '#424949',
    '#1abc9c', '#e8f8f5', '#d1f2eb', '#a3e4d7', '#76d7c4', '#48c9b0', '#1abc9c', '#17a589', 
'#148f77', '#117864', '#0e6251', '#16a085', '#e8f6f3', '#d0ece7', '#a2d9ce', '#73c6b6', 
'#45b39d', '#16a085', '#138d75', '#117a65', '#0e6655', '#0b5345', '#2ecc71', '#eafaf1', 
'#d5f5e3', '#abebc6', '#82e0aa', '#58d68d', '#2ecc71', '#28b463', '#239b56', '#1d8348', 
'#186a3b', '#27ae60', '#e9f7ef', '#d4efdf', '#a9dfbf', '#7dcea0', '#52be80', '#27ae60', 
'#229954', '#1e8449', '#196f3d', '#145a32', '#3498db', '#ebf5fb', '#d6eaf8', '#aed6f1', 
'#85c1e9', '#5dade2', '#3498db', '#2e86c1', '#2874a6', '#21618c', '#1b4f72', '#2980b9', 
'#eaf2f8', '#d4e6f1', '#a9cce3', '#7fb3d5', '#5499c7', '#2980b9', '#2471a3', '#1f618d', 
'#1a5276', '#154360', '#9b59b6', '#f5eef8', '#ebdef0', '#d7bde2', '#c39bd3', '#af7ac5', 
'#9b59b6', '#884ea0', '#76448a', '#633974', '#512e5f', '#8e44ad', '#f4ecf7', '#e8daef', 
'#d2b4de', '#bb8fce', '#a569bd', '#8e44ad', '#7d3c98', '#6c3483', '#5b2c6f', '#4a235a', 
'#34495e', '#ebedef', '#d6dbdf', '#aeb6bf', '#85929e', '#5d6d7e', '#34495e', '#2e4053', 
'#283747', '#212f3c', '#1b2631', '#2c3e50', '#eaecee', '#d5d8dc', '#abb2b9', '#808b96', 
'#566573', '#2c3e50', '#273746', '#212f3d', '#1c2833', '#17202a', '#f1c40f', '#fef9e7', 
'#fcf3cf', '#f9e79f', '#f7dc6f', '#f4d03f', '#f1c40f', '#d4ac0d', '#b7950b', '#9a7d0a', 
'#7d6608', '#f39c12', '#fef5e7', '#fdebd0', '#fad7a0', '#f8c471', '#f5b041', '#f39c12', 
'#d68910', '#b9770e', '#9c640c', '#7e5109', '#e67e22', '#fdf2e9', '#fae5d3', '#f5cba7', 
'#f0b27a', '#eb984e', '#e67e22', '#ca6f1e', '#af601a', '#935116', '#784212', '#d35400', 
'#fbeee6', '#f6ddcc', '#edbb99', '#e59866', '#dc7633', '#d35400', '#ba4a00', '#a04000', 
'#873600', '#6e2c00', '#e74c3c', '#fdedec', '#fadbd8', '#f5b7b1', '#f1948a', '#ec7063',
'#e74c3c', '#cb4335', '#b03a2e', '#943126', '#78281f', '#c0392b', '#f9ebea', '#f2d7d5',
'#e6b0aa', '#d98880', '#cd6155', '#c0392b', '#a93226', '#922b21', '#7b241c', '#641e16', 
'#ecf0f1', '#fdfefe', '#fbfcfc', '#f7f9f9', '#f4f6f7', '#f0f3f4', '#ecf0f1', '#d0d3d4', 
'#b3b6b7', '#979a9a', '#7b7d7d', '#bdc3c7', '#f8f9f9', '#f2f3f4', '#e5e7e9', '#d7dbdd', 
'#cacfd2', '#bdc3c7', '#a6acaf', '#909497', '#797d7f', '#626567', '#95a5a6', '#f4f6f6', 
'#eaeded', '#d5dbdb', '#bfc9ca', '#aab7b8', '#95a5a6', '#839192', '#717d7e', '#5f6a6a', 
'#4d5656', '#7f8c8d', '#f2f4f4', '#e5e8e8', '#ccd1d1', '#b2babb', '#99a3a4', '#7f8c8d',
 '#707b7c', '#616a6b', '#515a5a', '#424949',
    '#1abc9c', '#e8f8f5', '#d1f2eb', '#a3e4d7', '#76d7c4', '#48c9b0', '#1abc9c', '#17a589', 
'#148f77', '#117864', '#0e6251', '#16a085', '#e8f6f3', '#d0ece7', '#a2d9ce', '#73c6b6', 
'#45b39d', '#16a085', '#138d75', '#117a65', '#0e6655', '#0b5345', '#2ecc71', '#eafaf1', 
'#d5f5e3', '#abebc6', '#82e0aa', '#58d68d', '#2ecc71', '#28b463', '#239b56', '#1d8348', 
'#186a3b', '#27ae60', '#e9f7ef', '#d4efdf', '#a9dfbf', '#7dcea0', '#52be80', '#27ae60', 
'#229954', '#1e8449', '#196f3d', '#145a32', '#3498db', '#ebf5fb', '#d6eaf8', '#aed6f1', 
'#85c1e9', '#5dade2', '#3498db', '#2e86c1', '#2874a6', '#21618c', '#1b4f72', '#2980b9', 
'#eaf2f8', '#d4e6f1', '#a9cce3', '#7fb3d5', '#5499c7', '#2980b9', '#2471a3', '#1f618d', 
'#1a5276', '#154360', '#9b59b6', '#f5eef8', '#ebdef0', '#d7bde2', '#c39bd3', '#af7ac5', 
'#9b59b6', '#884ea0', '#76448a', '#633974', '#512e5f', '#8e44ad', '#f4ecf7', '#e8daef', 
'#d2b4de', '#bb8fce', '#a569bd', '#8e44ad', '#7d3c98', '#6c3483', '#5b2c6f', '#4a235a', 
'#34495e', '#ebedef', '#d6dbdf', '#aeb6bf', '#85929e', '#5d6d7e', '#34495e', '#2e4053', 
'#283747', '#212f3c', '#1b2631', '#2c3e50', '#eaecee', '#d5d8dc', '#abb2b9', '#808b96', 
'#566573', '#2c3e50', '#273746', '#212f3d', '#1c2833', '#17202a', '#f1c40f', '#fef9e7', 
'#fcf3cf', '#f9e79f', '#f7dc6f', '#f4d03f', '#f1c40f', '#d4ac0d', '#b7950b', '#9a7d0a', 
'#7d6608', '#f39c12', '#fef5e7', '#fdebd0', '#fad7a0', '#f8c471', '#f5b041', '#f39c12', 
'#d68910', '#b9770e', '#9c640c', '#7e5109', '#e67e22', '#fdf2e9', '#fae5d3', '#f5cba7', 
'#f0b27a', '#eb984e', '#e67e22', '#ca6f1e', '#af601a', '#935116', '#784212', '#d35400', 
'#fbeee6', '#f6ddcc', '#edbb99', '#e59866', '#dc7633', '#d35400', '#ba4a00', '#a04000', 
'#873600', '#6e2c00', '#e74c3c', '#fdedec', '#fadbd8', '#f5b7b1', '#f1948a', '#ec7063',
'#e74c3c', '#cb4335', '#b03a2e', '#943126', '#78281f', '#c0392b', '#f9ebea', '#f2d7d5',
'#e6b0aa', '#d98880', '#cd6155', '#c0392b', '#a93226', '#922b21', '#7b241c', '#641e16', 
'#ecf0f1', '#fdfefe', '#fbfcfc', '#f7f9f9', '#f4f6f7', '#f0f3f4', '#ecf0f1', '#d0d3d4', 
'#b3b6b7', '#979a9a', '#7b7d7d', '#bdc3c7', '#f8f9f9', '#f2f3f4', '#e5e7e9', '#d7dbdd', 
'#cacfd2', '#bdc3c7', '#a6acaf', '#909497', '#797d7f', '#626567', '#95a5a6', '#f4f6f6', 
'#eaeded', '#d5dbdb', '#bfc9ca', '#aab7b8', '#95a5a6', '#839192', '#717d7e', '#5f6a6a', 
'#4d5656', '#7f8c8d', '#f2f4f4', '#e5e8e8', '#ccd1d1', '#b2babb', '#99a3a4', '#7f8c8d',
 '#707b7c', '#616a6b', '#515a5a', '#424949'
]

from random import shuffle
shuffle(colormap)
colormap = np.array(colormap)

from sklearn.preprocessing import LabelEncoder
labels = data.tema+'-'+data.subtema
encoder = LabelEncoder()
y = encoder.fit_transform(list(labels))

In [ ]:
#groups_ids = y.argmax(1)
groups_ids = y

output_notebook()
plot_lda = bp.figure(plot_width=1400, plot_height=1100,
                     title=title,
                     tools="pan,wheel_zoom,box_zoom,reset,hover,previewsave",
                     x_axis_type=None, y_axis_type=None, min_border=1)

plot_lda.scatter(x=tsne_lda[:, 0], y=tsne_lda[:, 1],
                 color=colormap[groups_ids])

bp.show(plot_lda)

In [ ]:
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import numpy as np

init_notebook_mode(connected=True)
trace1 = go.Scatter3d(
    x=tsne_lda[:, 0],
    y= tsne_lda[:, 1],
    z= tsne_lda[:, 2],
    mode='markers',
    marker=dict(
        size=1,
        line=dict(
            color=colormap[groups_ids],
            width=0.5
        ),
        opacity=0.8
    )
)
data1 = [trace1]

layout = go.Layout(
    title=go.layout.Title(
        text='Plot Title',
        xref='paper',
        x=0
    ),
    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text='x Axis',
            font=dict(
                family='Courier New, monospace',
                size=18,
                color='#7f7f7f'
            )
        )
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text='y Axis',
            font=dict(
                family='Courier New, monospace',
                size=18,
                color='#7f7f7f'
            )
        )
    )
)


layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
fig = go.Figure(data=data1, layout=layout)
plot(fig, filename='simple-3d-scatter.html')

In [ ]:
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import numpy as np

import numpy as np

init_notebook_mode(connected=True)
x, y, z = np.random.multivariate_normal(np.array([0,0,0]), np.eye(3), 200).transpose()
trace1 = go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(
        size=12,
        line=dict(
            color='rgba(217, 217, 217, 0.14)',
            width=0.5
        ),
        opacity=0.8
    )
)

x2, y2, z2 = np.random.multivariate_normal(np.array([0,0,0]), np.eye(3), 200).transpose()
trace2 = go.Scatter3d(
    x=x2,
    y=y2,
    z=z2,
    mode='markers',
    marker=dict(
        color='rgb(127, 127, 127)',
        size=12,
        symbol='circle',
        line=dict(
            color='rgb(204, 204, 204)',
            width=1
        ),
        opacity=0.9
    )
)
data1 = [trace1, trace2]
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
fig = go.Figure(data=data1, layout=layout)
plot(fig, filename='simple-3d-scatter')

In [ ]:
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, plot, iplot

plot([go.Scatter(x=[1, 2, 3], y=[3, 1, 6])])